**[PT]** Português

---

**[EN]** English

# Bloco de experiências com linked data

---

# Notebook to experiment with linked data


## Exemplo de pesquisas

---

## Query examples

### Estudantes da Universidade de Coimbra nascidos antes do século XX na Wikidata

---

### Students of the University of Coimbra born before the 20th century in Wikidata

In [69]:
# Id of Universidade de Coimbra
uc_qid = 'Q368643'

# Property "educated at"
p_educated = 'P69'

# Property birth date
p_birth_date = 'P569'
# Property birth place
p_birth_place = 'P19'

# Property death date
p_death_date = 'P570'
# Property death place
p_death_place = 'P20'

# Coordinates of place
p_coordiantes = 'P625'

# country
p_country = 'P27'

# Biblioteca Nacional de Portugal ID
p_bnp_id = 'P1005'


# Install qwikidata library



In [70]:
!pip install qwikidata


### Obter todos os estudantes da UC, nascidos antes de 1900, com data e local de nascimento e morte, e id na Biblioteca Nacional, se disponível

E também: coordenadas do local de nascimento e outros nomes.

---

### Fetch all the students of UC born before 1900 with date and place of birth and deadth and Portuguese National Library ID

And also: coordinates of place of birth and aliases.

In [71]:
# for the method of getting the coordinates see https://en.wikibooks.org/wiki/SPARQL/WIKIDATA_Precision,_Units_and_Coordinates#Coordinates
all_query = """

SELECT DISTINCT ?student ?bnp_idLabel ?studentLabel ?altLabel ?countryLabel ?birthDateLabel ?birthPlace ?birthPlaceLabel ?longitude ?latitude ?deathDateLabel ?deathPlace ?deathPlaceLabel 
WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "pt". }
  {
    SELECT DISTINCT ?student ?altLabel ?bnp_id ?country ?birthDate ?birthPlace ?longitude ?latitude ?deathDate ?deathPlace WHERE {
      ?student p:P69 ?statement0.
      ?statement0 (ps:P69/(wdt:P279*)) wd:Q368643.
      ?student p:P569 ?statement_1.
      OPTIONAL { ?student p:P570/psv:P570/wikibase:timeValue ?deathDate.}
      OPTIONAL { ?student p:P27/ps:P27 ?country.}
      OPTIONAL { ?student p:P19/ps:P19 ?birthPlace.} 
      OPTIONAL { ?student p:P19/ps:P19/p:P625/psv:P625/wikibase:geoLongitude ?longitude.}
      OPTIONAL { ?student p:P19/ps:P19/p:P625/psv:P625/wikibase:geoLatitude ?latitude.}
      OPTIONAL { ?student p:P20/ps:P20 ?deathPlace.}
      OPTIONAL { ?student p:P1005/ps:P1005 ?bnp_id.}
      OPTIONAL { ?student skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "pt") }
      ?statement_1 psv:P569 ?statementValue_1.
      ?statementValue_1 wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?statementValue_1 wikibase:timeValue ?birthDate.
      FILTER(?birthDate< "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    ORDER BY ?studentLabel ?student
    LIMIT 1000
  }
}
"""

Query Wikidata

In [72]:
from qwikidata.sparql import return_sparql_query_results

results = return_sparql_query_results(all_query)
records = results['results']['bindings']


Show sample records

In [73]:
records[:2]

[{'student': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q94850304'},
  'altLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'Levy Maria Jordão de Paiva Manso'},
  'birthPlace': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q597'},
  'longitude': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '-9.139016'},
  'latitude': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '38.708042'},
  'deathPlace': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q597'},
  'bnp_idLabel': {'type': 'literal', 'value': '54707'},
  'studentLabel': {'xml:lang': 'pt',
   'type': 'literal',
   'value': 'Levy Maria Jordão'},
  'countryLabel': {'xml:lang': 'pt', 'type': 'literal', 'value': 'Portugal'},
  'birthDateLabel': {'type': 'literal', 'value': '1831-01-09T00:00:00Z'},
  'birthPlaceLabel': {'xml:lang': 'pt', 'type': 'literal', 'value': 'Lisboa'},
  'deathDateLabel': {'typ

Create dataframe from results

In [74]:
import pandas as pd


tuples = [(  
            student.get('student',{'value':None})['value'],
            student.get('studentLabel',{'value':None})['value'],
            student.get('altLabel',{'value':None})['value'],
            student.get('bnp_idLabel',{'value':None})['value'],
            student.get('birthPlaceLabel',{'value':None})['value'],
            student.get('birthPlace',{'value':None})['value'],
            student.get('longitude',{'value':None})['value'],
            student.get('latitude',{'value':None})['value'],
            student.get('birthDateLabel',{'value':'0000-00-00'})['value'][0:10], # keep just the date, the time is normally missing
            )
            for student in records]

students = pd.DataFrame(tuples, columns=['wikidata','name','alias','bnp_id','naturalidade','placeID','longitude','latitude','birth_date'])
students.drop_duplicates(subset=['wikidata','name','alias'], inplace=True)
print(students.info())
print("Number of unique wikidata entities:",len(students['wikidata'].unique()))
students.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 688 entries, 0 to 838
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wikidata      688 non-null    object
 1   name          688 non-null    object
 2   alias         462 non-null    object
 3   bnp_id        458 non-null    object
 4   naturalidade  651 non-null    object
 5   placeID       651 non-null    object
 6   longitude     649 non-null    object
 7   latitude      649 non-null    object
 8   birth_date    688 non-null    object
dtypes: object(9)
memory usage: 53.8+ KB
None
Number of unique wikidata entities: 429


,wikidata,name,alias,bnp_id,naturalidade,placeID,longitude,latitude,birth_date
0,http://www.wikidata.org/entity/Q94850304,Levy Maria Jordão,Levy Maria Jordão de Paiva Manso,54707,Lisboa,http://www.wikidata.org/entity/Q597,-9.139016,38.708042,1831-01-09
2,http://www.wikidata.org/entity/Q9619744,António Maria Vasco de Melo Silva César e Meneses,António de melo césar e meneses,537789,Lisboa,http://www.wikidata.org/entity/Q597,-9.139016,38.708042,1851-11-13
3,http://www.wikidata.org/entity/Q966168,Manuel Gonçalves Cerejeira,Gonçalves cerejeira,68690,Lousado,http://www.wikidata.org/entity/Q949303,-8.53028,41.351388888889,1888-11-29
4,http://www.wikidata.org/entity/Q9620291,António da Costa,António da Costa e Sousa de Macedo,105800,Lisboa,http://www.wikidata.org/entity/Q597,-9.139016,38.708042,1824-11-24
5,http://www.wikidata.org/entity/Q921705,Alexandre de Serpa Pinto,Alexandre Serpa Pinto,32584,Cinfães,http://www.wikidata.org/entity/Q861464,-8.0833333333333,41.066666666667,1846-04-20


#### Cruzar informação da Wikidate com o FAUC: comparar nome e local de nascimento, verificar datas

---

#### Record matching wikidata with FAUC: use name and place of birth, check dates



In [75]:
import warnings

from timelinknb.pandas import attribute_to_df
from timelinknb import get_db
from ucalumni.config import default_db
from IPython.display import display

db_spec = default_db
db = get_db(db_spec)

students['fauc_id'] = None

not_found = 0
match = 0
ambiguous = 0

def remove_particles(name):
    particles = ("de","da","e","das","dos","do")
    return " ".join([n for n in name.split() if n not in particles])


students.sort_values(['name','wikidata','alias','birth_date'], inplace=True)

match_list = []
duplicate_list = []
ambiguous_list = []

for i,row in students.iterrows():
    
    uri = row['wikidata']
    name = row['name']
  
    if uri in match_list:
        continue

    # fix specific name problems in wilkipedia
    if name == 'Samoa Barros':
        name = 'Samora Barros'
    elif name == 'Frei Custódio Alves Serrão':
        name = 'Custódio Alves Serrão'
    elif name == 'Levy Maria Jordão':
        name = 'Levi Maria Jordão'

    name_particles = remove_particles(name).split(" ")
    name_like = '%'.join(name_particles)
    name_like = '%'+name_like+'%'
    
    alias = row['alias']
    if alias is not None:
        alias_particles = remove_particles(alias).split(" ")
        alias_like = '%'.join(alias_particles)
        alias_like = '%'+alias_like+'%'

    if name is None and alias is None:
        warnings.warn(f"{uri} has null name and no alias, skipped.")
        continue
    
    birth_date = row['birth_date']
    print()
    if row['naturalidade'] is not None:
        place = row['naturalidade']
        aka = f"(aka {alias})" if alias is not None else ""
        print(f"Looking for {name} {aka} {place} {birth_date} {uri}")
        
        # get students with similar names from similar places
        candidates = attribute_to_df('nome-geografico',the_value='%'+place+'%',name_like=name_like,dates_in=(birth_date,'1920-00-00' ))
        
        if candidates.iloc[0].count() == 0 and alias is not None:
            # Try again with alias
            candidates = attribute_to_df('nome-geografico',the_value='%'+place+'%',name_like=alias_like,dates_in=(birth_date,'1920-00-00' ))

        if candidates.iloc[0].count() == 0:
            print("Not found")
            not_found = not_found + 1
        else:
            # some candidates were found
            display(candidates.head(5))
            
            # if just one we take it
            if len(candidates.index) == 1:
                print("Matched")
                fauc_id = candidates.iloc[0].name
                match = match + 1
                students.loc[i]['fauc_id'] = fauc_id
                match_list.append(uri)
            else:  # found more than one check if one name matches exactly
                hit = 0
                candidates_list = []
                for fid, row in candidates.iterrows():
                    if remove_particles(row['name']).lower() == remove_particles(name).lower():
                        hit = hit + 1
                        hit_id = fid
                        candidates_list.append(fid)
                if hit == 0 and alias is not None:
                    hit = 0
                    for fid, row in candidates.iterrows():
                        if remove_particles(row['name']).lower() == remove_particles(alias).lower():
                            hit = hit + 1
                            hit_id = fid      
                            candidates_list.append(fid)             
                if hit == 0:
                    print("Ambiguous: could not find match in candidates")
                    ambiguous = ambiguous + 1
                    ambiguous_list.append({'widata':uri,'name':name, 'fauc':list(candidates.index.unique())})
                elif hit > 0:
                    print("Matched")
                    fauc_id = hit_id  # note this 'name' is the index entry
                    match = match + 1
                    students.loc[i]['fauc_id'] = fauc_id
                    match_list.append(uri)
                    if hit > 1 :
                        duplicate_list.append({'widata':uri,'name':name, 'fauc':candidates_list})
                        warnings.warn("Possible duplicates: "+", ".join(candidates_list))

print(f"Matched {match} ambiguous {ambiguous} not found {not_found}")




Looking for Aarão Ferreira de Lacerda  Vila Nova de Paiva 1863-10-03 http://www.wikidata.org/entity/Q9559855
Not found

Looking for Abel Maria Jordão de Paiva Manso  Coimbra 1801-03-03 http://www.wikidata.org/entity/Q106271718
Not found

Looking for Abílio Augusto da Fonseca Pinto  Coimbra 1830-05-27 http://www.wikidata.org/entity/Q23820937


,name,sex,nome-geografico,nome-geografico.date,nome-geografico.obs
id,,,,,
245300,Abílio Augusto da Fonseca Pinto,m,Coimbra,1844-10-22,None


Matched


Looking for Adolfo Ferreira Loureiro (aka Adolfo Ferreira de Loureiro) São Bartolomeu 1836-12-12 http://www.wikidata.org/entity/Q19607365
Not found

Looking for Adriano António Canavarro Crispiniano da Fonseca  Marco de Canaveses 1884-11-08 http://www.wikidata.org/entity/Q16144296
Not found

Looking for Adriano Inácio Botelho  Angra do Heroísmo 1892-09-12 http://www.wikidata.org/entity/Q56072395


,name,sex,nome-geografico,nome-geografico.date,nome-geografico.obs
id,,,,,
186608,Adriano Inácio Botelho,m,Angra do Heroísmo,1910-11-02,Santa Luzia (Angra do Heroísmo)


Matched

Looking for Afonso Costa (aka Afonso Augusto Costa) Seia 1871-03-06 http://www.wikidata.org/entity/Q383401


...

Matched

Looking for Tomás de Sousa e Holstein Beck  Lisboa 1839-12-31 http://www.wikidata.org/entity/Q10383143
Not found

Looking for Toríbio de Mongrovejo  Mayorga 1538-11-26 http://www.wikidata.org/entity/Q467993


/var/folders/6m/_0n598297cj_bc2ql0rynnx00000gp/T/ipykernel_34979/2881094324.py:113: UserWarning: Possible duplicates: 144226, 144219
  warnings.warn("Possible duplicates: "+", ".join(candidates_list))


Not found

Looking for Venceslau de Lima (aka Venceslau de Sousa Pereira de Lima) Porto 1858-11-15 http://www.wikidata.org/entity/Q1392611
Not found

Looking for Venceslau de Lima (aka Wenceslau de Lima) Porto 1858-11-15 http://www.wikidata.org/entity/Q1392611
Not found

Looking for Venceslau de Lima (aka Wenceslau de Sousa Pereira de Lima) Porto 1858-11-15 http://www.wikidata.org/entity/Q1392611
Not found

Looking for Vicente Ferrer Neto de Paiva (aka Vicente Ferrer Neto Paiva) Lousã 1798-06-27 http://www.wikidata.org/entity/Q10390244
Not found

Looking for Vicente Ferrer Neto de Paiva (aka Vicente Ferrer de Neto Paiva) Lousã 1798-06-27 http://www.wikidata.org/entity/Q10390244
Not found

Looking for Victor Hugo de Azevedo Coutinho (aka Vitor Hugo de Azevedo Coutinho) Macau 1871-11-12 http://www.wikidata.org/entity/Q719020
Not found


Looking for Álvaro Rodrigues Azevedo (aka Alvaro Rodrigues d’Azevedo) Vila Franca de Xira 1825-03-20 http://www.wikidata.org/entity/Q17272838


,name,sex,nome-geografico,nome-geografico.date,nome-geografico.obs
id,,,,,
148851,Álvaro Rodrigues de Azevedo,m,Vila Franca de Xira,1843-10-02,None


Matched

Looking for Álvaro da Silva Sampaio  Angra do Heroísmo 1891-01-24 http://www.wikidata.org/entity/Q51882966
Not found
Matched 164 ambiguous 12 not found 378


Verificar quantos estudantes da Universidade de Coimbra na Wikidata foram identificados no FAUC

`fauc_id` non-null

---

Check how many Coimbra students in wikidata were matched with FAUC.

`fauc_id` non-null

In [78]:
students.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 688 entries, 40 to 293
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   wikidata      688 non-null    object
 1   name          688 non-null    object
 2   alias         462 non-null    object
 3   bnp_id        458 non-null    object
 4   naturalidade  651 non-null    object
 5   placeID       651 non-null    object
 6   longitude     649 non-null    object
 7   latitude      649 non-null    object
 8   birth_date    688 non-null    object
 9   fauc_id       164 non-null    object
dtypes: object(10)
memory usage: 75.3+ KB


In [79]:
students.head(20)

,wikidata,name,alias,bnp_id,naturalidade,placeID,longitude,latitude,birth_date,fauc_id
40,http://www.wikidata.org/entity/Q9559855,Aarão Ferreira de Lacerda,None,103998,Vila Nova de Paiva,http://www.wikidata.org/entity/Q1003146,-7.7333333333333,40.85,1863-10-03,None
675,http://www.wikidata.org/entity/Q106271718,Abel Maria Jordão de Paiva Manso,None,None,Coimbra,http://www.wikidata.org/entity/Q45412,-8.4288888888889,40.211111111111,1801-03-03,None
576,http://www.wikidata.org/entity/Q23820937,Abílio Augusto da Fonseca Pinto,None,314551,Coimbra,http://www.wikidata.org/entity/Q45412,-8.4288888888889,40.211111111111,1830-05-27,245300
9,http://www.wikidata.org/entity/Q9561614,Abílio Beça,None,None,None,None,None,None,1856-08-20,None
538,http://www.wikidata.org/entity/Q19607365,Adolfo Ferreira Loureiro,Adolfo Ferreira de Loureiro,83492,São Bartolomeu,http://www.wikidata.org/entity/Q1026333,-8.43054,40.20848,1836-12-12,None
603,http://www.wikidata.org/entity/Q16144296,Adriano António Canavarro Crispiniano da Fonseca,None,None,Marco de Canaveses,http://www.wikidata.org/entity/Q188050,-8.15,41.183333333333,1884-11-08,None
174,http://www.wikidata.org/entity/Q56072395,Adriano Inácio Botelho,None,15765,Angra do Heroísmo,http://www.wikidata.org/entity/Q193677,-27.215277777778,38.655833333333,1892-09-12,186608
345,http://www.wikidata.org/entity/Q383401,Afonso Costa,Afonso Augusto Costa,34862,Seia,http://www.wikidata.org/entity/Q76017,-7.70245,40.42203,1871-03-06,198981
341,http://www.wikidata.org/entity/Q383401,Afonso Costa,Afonso Augusto da Costa,34862,Seia,http://www.wikidata.org/entity/Q76017,-7.70245,40.42203,1871-03-06,None
253,http://www.wikidata.org/entity/Q4689383,Afonso Mendes,None,None,Santo Aleixo da Restauração,http://www.wikidata.org/entity/Q2223673,-7.15,38.066666666667,1579-06-28,None


In [81]:
students.to_csv('../inferences/wikidata/students_wikidata_matched.csv',sep=',')

#### Processar os nomes não ligados no primeiro passo

---

#### Processing unliked names in the first pass



In [82]:
unlinked = students[students['fauc_id'].isnull()]
unlinked[['name','alias','naturalidade','birth_date','wikidata']].head(30).fillna("")

,name,alias,naturalidade,birth_date,wikidata
40,Aarão Ferreira de Lacerda,,Vila Nova de Paiva,1863-10-03,http://www.wikidata.org/entity/Q9559855
675,Abel Maria Jordão de Paiva Manso,,Coimbra,1801-03-03,http://www.wikidata.org/entity/Q106271718
9,Abílio Beça,,,1856-08-20,http://www.wikidata.org/entity/Q9561614
538,Adolfo Ferreira Loureiro,Adolfo Ferreira de Loureiro,São Bartolomeu,1836-12-12,http://www.wikidata.org/entity/Q19607365
603,Adriano António Canavarro Crispiniano da Fonseca,,Marco de Canaveses,1884-11-08,http://www.wikidata.org/entity/Q16144296
341,Afonso Costa,Afonso Augusto da Costa,Seia,1871-03-06,http://www.wikidata.org/entity/Q383401
253,Afonso Mendes,,Santo Aleixo da Restauração,1579-06-28,http://www.wikidata.org/entity/Q4689383
612,Aires Jácome Correia,,Lisboa,1882-08-09,http://www.wikidata.org/entity/Q16144891
674,Alberto Teles de Utra Machado,,Angra do Heroísmo,1840-01-24,http://www.wikidata.org/entity/Q16487343
91,Albino dos Reis,Albino Soares Pinto dos Reis,Oliveira de Azeméis,1888-09-30,http://www.wikidata.org/entity/Q9594728


Current misses:
* Aarão Ferreira de Lacerda  Vila Nova de Paiva 1863-10-03 http://www.wikidata.org/entity/Q9559855
  * FAUC place of birth Touro https://linode.timelink-mhk.net/mhk/ucalumni/id/265860 Wikidata: Vila Nova de Paiva
* Abel Maria Jordão de Paiva Manso		Coimbra	1801-03-03	http://www.wikidata.org/entity/Q106271718
  * FAUC has him as father: https://linode.timelink-mhk.net/mhk/ucalumni/id/193376-pai 
  * And him as student: https://linode.timelink-mhk.net/mhk/ucalumni/id/193254 but gives place name as "Buarcos". Father name match genealogical information at: 
        https://gw.geneanet.org/aqf?lang=fr&n=manso&oc=0&p=abel+maria+jordao+de+paiva 
  * See https://github.com/joaquimrcarvalho/fauc1537-1919/issues/13
* Abílio Beça			1856-08-20	http://www.wikidata.org/entity/Q9561614
  * 
* Looking António Augusto de Sousa 1883-12-10 http://www.wikidata.org/entity/Q23905297
  * Not found in FAUC  
* Alfredo Augusto de Soveral Martins Almacave 1869-06-07 http://www.wikidata.org/entity/Q1636522
  * Place name do not match. FAUC Lamego, Wikidata Almacave, which is parish
* Bernardo de Brito Almeida 1569-08-20 http://www.wikidata.org/entity/Q3638813
  * FAUC no birth place
* António Ginestal Machado Almeida 1874-05-03 http://www.wikidata.org/entity/Q357489
  * Not found in FAUC
* Levy Maria Jordão Lisboa 1831-01-09 http://www.wikidata.org/entity/Q94850304
  * Name in FAUC is Levi Maria Jordão
* António Maria Vasco de Melo Silva César e Meneses Lisboa 1851-11-13 http://www.wikidata.org/entity/Q9619744
  * FAUC name = António Vasco de Melo Silva César e Menezes
* Manuel Gonçalves Cerejeira Lousado 1888-11-29 http://www.wikidata.org/entity/Q966168
  * FAUC place of birth Lousada
* António de Sousa de Macedo Porto 1606-12-15 http://www.wikidata.org/entity/Q8202081
  * FAUC possible https://linode.timelink-mhk.net/mhk/ucalumni/id/183307 but place of birth does not match
* Pedro de Sousa Holstein, Duque de Palmela Turim 1781-05-08 http://www.wikidata.org/entity/Q982358
  * Homonym in https://linode.timelink-mhk.net/mhk/ucalumni/id/190226 but dates don't match also no one 
    born in Turim with similar name in FAUC
* António Pinto de Magalhães Aguiar Constance 1834-06-23 http://www.wikidata.org/entity/Q9620002
  * matches António Pinto de Magalhães Aguiar id: 141234 but place of birth does not match FAUC = Santa Eulália  
    de Constância
* Gregório de Matos, http://www.wikidata.org/entity/Q983565 corresponds to Gregório de Matos Guerra id: 180656 vide https://linode.timelink-mhk.net/mhk/ucalumni/id/171909
  * Misses because wikidata gives place of Birth as Salvador and FAUC gives Baía
'Anselmo José Braamcamp' 'Anselmo José Braancamp'

In [48]:
ambiguous_list

[['187197', '183723'],
 ['236173', '226251'],
 ['198631', '292664'],
 ['198631', '292664'],
 ['198631', '292664'],
 ['198631', '292664'],
 ['201877', '159082', '194045', '188555', '176794'],
 ['201877', '159082', '194045', '188555', '176794'],
 ['172342', '250289', '172343'],
 ['210673', '132696'],
 ['210673', '132696'],
 ['210673', '132696']]

2022-05-18 : 454 results stored in [inferences/wikidata](../inferences/wikidata/query.json)

In [1]:

query = """

    SELECT DISTINCT ?student 
     WHERE {
      ?student p:P69 ?educatedAt.
      ?educatedAt (ps:P69/(wdt:P279*)) wd:Q368643.
      ?student p:P569 ?birthDateStmt.
      ?birthDateStmt psv:P569 ?birthDateValue.
      ?birthDateValue wikibase:timePrecision ?precision_1.
      FILTER(?precision_1 >= 11 )
      ?birthDateValue wikibase:timeValue ?P569_1.
      FILTER(?P569_1 < "+1900-01-01T00:00:00Z"^^xsd:dateTime)
    }
    LIMIT 5

"""

In [2]:
!pip install lxml

You should consider upgrading via the '/Users/jrc/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
from datetime import datetime

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from qwikidata.sparql import return_sparql_query_results

from lxml import etree

# Prepare to fetch records from Portuguese National Library



xsl_file = '../extras/bnp/visbd-fauc.xsl'
xsl = etree.parse(xsl_file) 

# Things we would like to known about the students
claims_of_interest = ['P69','P1559','P1447','P27','P569','P570','P106','P19','P20',
                      'P22','P25','p97', 
                      'P973',                      # described at URL
                      'P1005','P102','P39','P166','P213','P214']
                      
claims_ignored = set()
prop_labels = {}
cache = {}
results = return_sparql_query_results(query)

# https://qwikidata.readthedocs.io/en/stable/entity.html
for result in results['results']['bindings']:
    uri = result['student']['value']
    qid = uri.split("/")[-1]  # last item in the path
    result_dict = get_entity_dict_from_api(qid)
    std = WikidataItem(result_dict)

    print(std.entity_id,std.get_label())
    for aka in std.get_aliases(lang='pt'):
        print(f"{' '*2} aka:",aka)

    claim_groups = std.get_claim_groups()


    claim_id_list = sorted([c for c in claim_groups.keys() if c in claims_of_interest])
    if 'P1005' in claim_id_list:     # we move the property related to published works to the end
        claim_id_list.remove('P1005')
        claim_id_list = claim_id_list + ['P1005']


    for claim_id in claim_id_list:
        if claim_id in prop_labels.keys():
            propLabel = prop_labels[claim_id]
        else:
            prop_dict = get_entity_dict_from_api(claim_id)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[claim_id] = propLabel

        claim_group = claim_groups[claim_id]
        print(f"{' '*2} {claim_id} {propLabel}({len(claim_group)})")
        for claim in claim_group:
            snak = claim.mainsnak
            if snak.snak_datatype == 'wikibase-item':
                qid = snak.datavalue.value["id"]
                entity = WikidataItem(get_entity_dict_from_api(qid))
                print(f"{' '*6}{qid} {entity.get_label(lang='pt')}")
            elif snak.value_datatype == 'string':
                print(f"{' '*6} {snak.datavalue.value}") 
            elif snak.value_datatype == 'time':
                svalue = snak.datavalue.value['time']
                try:
                    date_value = datetime.strptime(svalue,"+%Y-%m-%dT%H:%M:%SZ")
                    print(f"{' '*6} {date_value}")  
                except ValueError as e:
                    print("Problem converting date, probably date incomplete: "+svalue,e)
            else:
                snak_info = (claim.rank,snak.snaktype,snak.snak_datatype,snak.value_datatype)  
                print(f"{' '*6}{snak_info}")
            
            if claim_id == 'P1005': 
                bnp_id = snak.datavalue.value
                url = f"http://urn.bn.pt/bibliografia/unimarc/xml?id={bnp_id}"
                records = etree.parse(url)
                transform = etree.XSLT(xsl)
                print(str(transform(records)))

            for pid, quals in claim.qualifiers.items():
                prop = WikidataProperty(get_entity_dict_from_api(pid))
                for qual in quals:
                    if qual.snak.snaktype != "value":
                        continue
                    else:
                        print(f"{' '*9}{prop.get_label()} {qual.snak.datavalue}")

    print(f"{' '*3}Other properties:",set(claim_groups.keys())-set(claims_of_interest) )
    claims_ignored = claims_ignored.union(set(claim_groups.keys())-set(claims_of_interest))
    print()

print()
if len(claims_ignored) > 0:
    print("Properties present in the students and ignored in this list")
    for ignored in sorted(claims_ignored):
        if ignored in prop_labels.keys():
            propLabel = prop_labels[prop]
        else:
            prop_dict = get_entity_dict_from_api(ignored)
            propLabel = prop_dict['labels']['en']['value']
            prop_labels[prop] = propLabel  
        print(f"   {ignored:<6s}: {propLabel}")



Q61755108 Manuel Augusto Martins
   P106 occupation(1)
      Q82955 político
   P19 place of birth(1)
      Q2078801 Santa Maria Maior
   P20 place of death(1)
      Q25444 Funchal
   P27 country of citizenship(1)
      Q45 Portugal
   P569 date of birth(1)
       1867-04-10 00:00:00
   P570 date of death(1)
       1936-04-04 00:00:00
   P69 educated at(1)
      Q368643 Universidade de Coimbra
         academic major WikibaseEntityId(id=Q7748)
   Other properties: {'P735', 'P21', 'P1412', 'P18', 'P2671', 'P103', 'P31', 'P734'}

Q16917757 Francisco Correia de Mendonça
   P106 occupation(1)
      Q16533 juiz
   P19 place of birth(1)
      Q732548 Lagos
   P27 country of citizenship(1)
      Q45 Portugal
   P569 date of birth(1)
       1832-01-03 00:00:00
   P570 date of death(1)
       1899-08-04 00:00:00
   P69 educated at(1)
      Q368643 Universidade de Coimbra
   Other properties: {'P735', 'P21', 'P1412', 'P2671', 'P103', 'P31'}

Q17272838 Álvaro Rodrigues Azevedo
   aka: José Rodrig

## Lugares

---

## Places


In [ ]:
place = "Soure"

In [ ]:
pquery = """

SELECT DISTINCT ?location ?locationName ?instanceOf ?instanceOfLabel ?insideLabel ?inside2Label ?coordinates ?geonamesID WHERE {
                        ?location ?label '$place'@pt .  
                        ?location wdt:P17 wd:Q45 .
                        ?location rdfs:label ?locationName .
                        ?location wdt:P31 ?instanceOf .
                        ?location wdt:P625 ?coordinates .
                        ?location wdt:P131 ?inside .

                        OPTIONAL {

                          ?inside wdt:P131 ?inside2 .
                          ?location wdt:P1566 ?geonamesID .
                        }
                        FILTER(lang(?locationName) = "pt") 
                        { SERVICE wikibase:label { bd:serviceParam wikibase:language "pt". }}
 
                        }
"""

#### Encontrar um lugar na wikipedia

---

### Find a place in Wikipedia

In [ ]:
!pip install mkwikidata


In [ ]:
import mkwikidata
import pandas as pd

place = 'Soure'

query_result = mkwikidata.run_query(pquery, params={'place':place})
results = query_result['results']['bindings']
table = [{'name':loc['locationName']['value'],
          'type':loc['instanceOfLabel']['value'],
          'uri':loc['location']['value'],
          'inside':loc['insideLabel']['value'],
          'inside2':loc.get('inside2Label',{'value':None})['value'],
          'geonamesID':loc.get('geonamesID',{'value':None})['value'],
          
          } for loc in results]

pd.DataFrame(table).sort_values('uri')

,name,type,uri,inside,inside2,geonamesID
2,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Região de Coimbra,8011895
4,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Região de Coimbra,8011895
5,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Coimbra,8011895
8,Soure,freguesia de Portugal,http://www.wikidata.org/entity/Q1977811,Soure,Coimbra,8011895
0,Soure,município de Portugal,http://www.wikidata.org/entity/Q543993,Coimbra,Portugal,2733851
1,Soure,vila de Portugal,http://www.wikidata.org/entity/Q543993,Coimbra,Portugal,2733851
3,Soure,vila de Portugal,http://www.wikidata.org/entity/Q543993,Região de Coimbra,None,None
7,Soure,município de Portugal,http://www.wikidata.org/entity/Q543993,Região de Coimbra,None,None
6,Soure,vila de Portugal,http://www.wikidata.org/entity/Q97597938,Soure,None,None


In [ ]:
from timelinknb import current_time,current_machine, get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)
print(current_machine,current_time,f'db={db_spec}')

imac-jrc.local 2022-05-18 17:43:33.378466 db=('sqlite', 'fauc.db')


### Localizar os lugares mais infrequentes no FAUC

---

### Locate the less frequent places in the FAUC

In [ ]:
from timelinknb.pandas import attribute_values


# Get list of different values and the number of occurences
naturalidades = attribute_values('naturalidade')
small_places = naturalidades[naturalidades['count']<10].sort_values('count', ascending=False).head(100)

Test with a small number

In [ ]:
from IPython.display import display
import time

for place in small_places.index.values[:10]:
    print("Looking for:", place)
    try:
        query_result = mkwikidata.run_query(pquery, params={'place':place})
    except Exception as e:
        print(e)
    results = query_result['results']['bindings']
    if len(results) > 0:
        table = [{'name':loc['locationName']['value'],
                'type':loc['instanceOfLabel']['value'],
                'uri':loc['location']['value'],
                'inside':loc['insideLabel']['value'],
                'inside2':loc.get('inside2Label',{'value':None})['value'],
                'geonamesID':loc.get('geonamesID',{'value':None})['value'],
                
                } for loc in results]
        candidates = pd.DataFrame(table)
        display(candidates)
        geonames = candidates['geonamesID'].unique()
        for geo in geonames:
            print(f"   https://www.geonames.org/{geo}")
    else:
        print("   not found")
        
    time.sleep(1)  # respect wikidata

Looking for: Alcofra


,name,type,uri,inside,inside2,geonamesID
0,Alcofra,freguesia de Portugal,http://www.wikidata.org/entity/Q1021445,Vouzela,Viseu,8014009
1,Alcofra,freguesia de Portugal,http://www.wikidata.org/entity/Q1021445,Vouzela,Comunidade Intermunicipal Viseu Dão Lafões,8014009


   https://www.geonames.org/8014009
Looking for: Lumiar


,name,type,uri,inside,inside2,geonamesID
0,Estação Lumiar,estação subterrânea,http://www.wikidata.org/entity/Q374170,Lisboa,None,None
1,Estação Lumiar,estação de metropolitano,http://www.wikidata.org/entity/Q374170,Lisboa,None,None
2,Lumiar,freguesia de Portugal,http://www.wikidata.org/entity/Q924723,Lisboa,Lisboa,8012475


   https://www.geonames.org/None
   https://www.geonames.org/8012475
Looking for: Seixo de Ansiães


,name,type,uri,inside,inside2,geonamesID
0,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Trás-os-Montes e Alto Douro,8014442
1,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Bragança,8014442
2,Seixo de Ansiães,freguesia de Portugal,http://www.wikidata.org/entity/Q1011237,Carrazeda de Ansiães,Douro,8014442


   https://www.geonames.org/8014442
Looking for: Santa Cristina de Figueiró


,name,type,uri,inside,inside2,geonamesID
0,Santa Cristina de Figueiró,localidade,http://www.wikidata.org/entity/Q1023987,Amarante,Porto,8012656
1,Santa Cristina de Figueiró,freguesia de Portugal,http://www.wikidata.org/entity/Q1023987,Amarante,Douro Litoral,8012656
2,Santa Cristina de Figueiró,localidade,http://www.wikidata.org/entity/Q1023987,Amarante,Douro Litoral,8012656
3,Santa Cristina de Figueiró,freguesia de Portugal,http://www.wikidata.org/entity/Q1023987,Amarante,Porto,8012656


   https://www.geonames.org/8012656
Looking for: Sandelgas
   not found
Looking for: Salvaterra
   not found
Looking for: Rio Bom
   not found
Looking for: Ribeira de Homem
   not found
Looking for: Quintãs
   not found
Looking for: Proença
   not found


## Biblioteca Nacional de Portugal

---

## Portuguese National Library



Get all the bibiographic records of a student with P1005 id.



In [94]:
from lxml import etree

bnp_id = "92687"
url = f"http://urn.bn.pt/bibliografia/unimarc/xml?id={bnp_id}"

xsl_file = '../extras/bnp/visbd-fauc.xsl'
xsl = etree.parse(xsl_file) 
records = etree.parse(url)
transform = etree.XSLT(xsl)

print(str(transform(records)))





Socialismo libertario e anarchismo : história e doutrina / Silva Mendes. - [S.l. : s.n.], 1896. - 369 p. ; 22 cm
http://id.bnportugal.gov.pt/bib/catbnp/418597

Sobre filosofia / Manuel da Silva Mendes. - Macau : Leal Senado, [1915]. - 169 p. : il. ; 21 cm
http://id.bnportugal.gov.pt/bib/catbnp/1579131

Socialismo libertário ou anarchismo : história e doutrina / Manuel da Silva Mendes. - ed. fac-simil. - Lisboa : Letra Livre, D.L. 2006. - 369 p. : fac-simil. ; 24 cm. - Ed. fac-simil., Lisboa, 1896. - ISBN 989-95114-0-4. - ISBN 978-989-95114-0-8
http://id.bnportugal.gov.pt/bib/catbnp/1563060

Manuel da Silva Mendes : a instrução pública em Macau ; org. e introd. António Aresta. - Macau : Direcção dos Serviços de Educação e Juventude, 1996. - 96, [5] p., [7] p. il. : il. ; 24 cm. - (Educação memórias ; 3). - ISBN 972-8091-32-X
http://id.bnportugal.gov.pt/bib/catbnp/987433

Arte chinesa : (colectânea de artigos) / Manuel da Silva Mendes. - [Macau] : Edição do Leal Senado de Macau, 1983. -

In [16]:
recs = records.getroot()
recs.tag

'{http://www.bn.pt/standards/metadata/marcxml/1.0/}collection'

In [92]:
marcxml_ns = "http://www.bn.pt/standards/metadata/marcxml/1.0/"
nsmap = {None: marcxml_ns}
for rec in recs:
    print("Record:")
    for cf in rec.findall("controlfield",namespaces=nsmap):
        print(cf.get('tag'),cf.text)
    for df in rec.findall("datafield",namespaces=nsmap):
        print(f"{df.get('tag'):3s} {df.get('ind1'):1s}{df.get('ind2'):1s}")
        for sf in list(df):
            print(f"   ${sf.get('code')} {sf.text}",)
print()


Record:
001 418597
003 http://id.bnportugal.gov.pt/bib/catbnp/418597
005 19981231000000.0
095   
   $a PTBN00447303
100   
   $a 19930417d1896    m  y0pory0103    ba
101 0 
   $a por
102   
   $a PT
200 0 
   $a Socialismo libertario e anarchismo
   $e história e doutrina
   $f Silva Mendes
210   
   $a [S.l.
   $c s.n.],
   $d 1896
215   
   $a 369 p.
   $d 22 cm
500 10
   $a Socialismo libertário ou anarquismo
700  1
   $a Mendes,
   $b Silva,
   $f 1867-1931
   $3 92687
801  0
   $a PT
   $b BN
   $g RPC
966   
   $l BN
   $m FGMON
   $s S.C. 3676 V.
966   
   $l BN
   $m FGMON
   $s S.C. 11055 P.
997   
   $a BIBEOD
Record:
001 1579131
003 http://id.bnportugal.gov.pt/bib/catbnp/1579131
100   
   $a 20070321d1915    m  y0pory0103    ba
101 0 
   $a por
102   
   $a MO
105   
   $a a   z   001yy
106   
   $a r
200 1 
   $a Sobre filosofia
   $f Manuel da Silva Mendes
210   
   $a Macau
   $c Leal Senado,
   $d [1915]
215   
   $a 169 p.
   $c il.
   $d 21 cm
675   
   $a 22Taoísmo
  

In [10]:
recs = records['collection']

cfields = recs['record'][0]['controlfield']
dfields = recs['record'][0]['datafield']
cfields


TypeError: 'lxml.etree._ElementTree' object is not subscriptable